## LFU Cache
**"Least Frequently Used"**: In this strategy you keep track of how often an item is used (written OR accessed), and get rid the items that are used least frequently when the cache's capacity is reached.

Before getting into the implementation, lets just experiment a bit!


In [ ]:
lfu = LFUCache(3)
lfu.put(1,1)
lfu.put(2,2)
lfu.put(3,3)

see_inside(lfu)

In [ ]:
lfu.put(2,4)
see_inside(lfu)

In [ ]:
lfu.put(5,5)
see_inside(lfu)

In [ ]:
lfu.put(5,6)
see_inside(lfu)

In [ ]:
lfu.put(3,7)
see_inside(lfu)

In [ ]:
lfu.put(8,8)
see_inside(lfu)


### MinHeap

*Traditionally*, an LFU cache is implemented with a **MinHeap** data structure because it handles insertion, deletion, and update in **O(log n)** time complexity:

A **MinHeap** is a Tree structure with the following properties.
1. Complete: (All levels are completely filled before adding additional levels, and levels are filled from left to right). This allows a Min Heap to be stored in an array.
2. The key at root must be minimum among all keys present in the Heap, and the same property must be recursively true for all nodes in Binary Tree.

**Note:** that there is no guaruntee that nodes on the same level are sorted from left to right across different branches!

![binary-heap](img/binaryheap.png)

## Dictionary + Linked Lists Implementation

A more modern approach to the LFU cache uses a combination of a **hash table** (dictionary) and collection of **linked-lists** to reduce reduce read-write time to **O(1)** operations!

Detailed explanation (for Java) can be found here:
https://ieftimov.com/post/when-why-least-frequently-used-cache-implementation-golang/


For an LFU Cache, we can use the following data structures:

- "**frequency_of_key**" is a normal dictionary that maps one key to the *number of times it was used* (**frequency**).

- "**keys_with_frequency**" is a normal dictionary that maps one frequency to an **OrderdDict** that holds *all the keys with that frequency*.

- The **OrderedDict(s)** then map each key to its respective value

- Finally, we keep track of the "**lowest_frequency**" (int) at all times, so we know what to delete next

With these two dictionaries (plus embedded OrderedDicts):

- Given one key, we can find its frequency

- When there are many items of *same frequency*, the **OrderedDict** tells us the *order in which the items were added*, so *the first item put in will be the first one to pop out* (like a **queue**).

![lfu-backbone](img/lfu-backbone-linked-lists.png)

### Algorithm

**Get an item from the cache**:

1. Look for the key in the "**frequency_of_keys**" dictionary.

  - If it is not found, there is no item, so return None (or -1)

2. If the item is found, get the frequency that the key was accessed

3. With the correct frequency, get the OrderedDict which holds all of the keys/values with that frequency from the "**frequency_of_key**" dictionary 


**Add an item to the cache**:

- Check to see if the key is already there.

  - If it is, update its value, and increase its frequency count. \*
  
  - If the key is not present, add the item, and set its frequency to 1. \*

    - If we added an item, check to see if we are already at capacity.  
    
      - If not, increase the current_size counter. 
      
      - If so, we have to evict something!

\* **Note**: Whenever you change an item's frequency, you will need to re-arrange the linked-list structure

- Remove the node from its current linked list

  - If that was the last node in that linked list, then delete that linked list, and that frequency from the frequencies dictionary
  
- Add the node to the linked list at the next higher frequency (at the front of the linked list)

  - If there was not already an entry for that frequency in the frequencies dictionary, you may need to add it, and create a new linked list for that frequency! 

This Diagram may help show what is happening:

![lfu-insertion](img/new-frequency-entry-insertion.png)



### Evicting an item:

1. Find the minimum frequency through hashmap values. 

2. Find the first occurence of that frequency in ordereddict, remove that element.

3. If that was the last node in that linked list, delete the linked list, and remove that frequency from the frequencies dictionary

https://leetcode.com/problems/lfu-cache/discuss/369104/Python-two-dicts-explanation

In [ ]:
class LFUCache:
    def __init__(self, capacity: int):
        # We must pre-define the capacity of the LFU cache on creation
        self.capacity=capacity
        self.lowest_frequency=None
        self.frequency_of_key={}
        
        # the defaultdict class takes one argument, which is a "factory"
        # telling what kind of objects to populate the dictionary by default
        # this way if we try to add values to a previously non-existent key
        # it will automatically create an OrdreredDict object to hold them!
        self.keys_with_frequency = collections.defaultdict( collections.OrderedDict )

    def get(self, key: int) -> int:
        
        if key not in self.frequency_of_key:
            # key not found
            return -1
        # otherwise, we should have it somewhere!
        
        # find out how many times this key has been accessed before
        freq = self.frequency_of_key[key]
        
        # "keys_with_frequency[freq]" returns an OrderedDict, then we give it a key to get its value
        val = self.keys_with_frequency[freq][key]
        
        # we delete this key/value pair from the OrderedDict at its previous frequency, 
        # because it will no longer have that frequency anymore!
        # We will remake it below at the next higher frequency
        del self.keys_with_frequency[freq][key]
        
        
        # Check to see if that was the last remaining entry in the dictionary at that frequency
        if not self.keys_with_frequency[freq]:
            
            # if it was the last entry, and it was also the lowest frequency
            if freq==self.lowest_frequency:
                
                # the new lowest frequency will now be one higher than before!
                self.lowest_frequency+=1
            
            # since we no longer have any items with that frequency, 
            # we can remove that frequency from the dictionary
            del self.keys_with_frequency[freq]
            
        # now that key has a higher frequency than before
        self.frequency_of_key[key]=freq+1
        
        # so we add a new frequency to the keys_with_frequency dict, 
        # which automatically creates a new OrderedDict (default constructor)
        # then we put a new key/value pair into that OrderedDict (which automatically goes to the head)
        self.keys_with_frequency[freq+1][key]=val
        
        # finally, we return the value that was asked for
        return val

    def put(self, key: int, value: int) -> None:
        
        # if the capcity is zero, we have no room to cache anything!
        if self.capacity<=0:
            return
        
        # if the key is already present
        if key in self.frequency_of_key:
            
            # get the frequency of that key
            freq=self.frequency_of_key[key]
            
            # then use that frequency to get the OrderedDict of key/value pairs with that frequency.  
            # then overwrite the value for that key to that OrderedDict.
            self.keys_with_frequency[freq][key]=value
            
            # calling the get method will take care of changes to the frequency and ordering
            self.get(key)
            return
        
        # if we are already at max capacity
        # (though it seems like this only counts the number of different frequencies
        # rather than the total number of items...)
        if self.capacity==len(self.frequency_of_key):
            
            # pop the key and value of the oldest item from the OrderedDict at the lowest frequency
            # This removes that key/value pair form the OrderedDict
            # Note: the "last" flag in OrderedDict asks if you want the last item added, 
            # but in this case we want the first item added so we set it to False
            delkey,delval=self.keys_with_frequency[self.lowest_frequency].popitem(last=False)
            
            # Since we got rid of that item, we no longer need to keep track of its frequency
            del self.frequency_of_key[delkey]
        
        # if the item is new, it must have a frequency of 1 (first time accessed)
        self.frequency_of_key[key]=1
        
        # get/make the OrderedDict for frequency 1, and add the new key/value pair to it
        self.keys_with_frequency[1][key]=value
        
        # reset lowest_frequency back to 1 (since we added a new item)
        self.lowest_frequency=1


## Top-voted solution on Leetcode
https://leetcode.com/problems/lfu-cache/discuss/207673/Python-concise-solution-**detailed**-explanation%3A-Two-dict-%2B-Doubly-linked-list

### Explanation

**class Node**:
- key: int
- value: int
- freq: int
- prev: Node
- next: Node

Each node keeps track of a single key/value pair, its own frequency, as well as pointers to the next and previous nodes.

**class DLinkedList**:
- _sentinel: Node
- size: int
- append(node: Node) -> None
- pop(node: Node) -> Node

Individual nodes are collected into doubly-linked lists, which keep track of the number of nodes in that list, and provide convenient append and pop methods to add and remove elements.
- **Note**: the *sentinel* node sits at the start/end of the LL, so you can quickly find the head/tail


**class LFUCache**:
- _node: dict[key: int, node: Node]
- _freq: dict[freq: int, lst: DlinkedList]
- _minfreq: int
- get(key: int) -> int
- put(key: int, value: int) -> None

*LFUCache._node* maps a *key* to a single node, so we can retrieve that node in O(1) time.

*LFUCache._freq* maps a frequency (*freq*) to a Doubly Linked List, 
where all nodes in the DLinkedList have the same frequency. 
Moreover, each new node will be always inserted in the head (indicating most recently used).

*LFUCache._minfreq* keeps track of the minimum frequency of across all nodes in this cache, such that the DLinkedList with the min frequency can always be retrieved in O(1) time.


![lfu-topvote](img/lfu-topvote.png)



### Algorithm:

**get(key)**
- query the node by calling self._node[key]
- find the frequency by checking node.freq (call this *f*)
- find the DLinkedList that this node was in previously, by calling self._freq[f] (call this *dl_prev*)
- pop this node (removing it from the DLinkedList)
- if *dl_prev* is empty and self._minfreq == f, update self._minfreq to f+1.
- increase the node's internal frequency counter: node.freq --> f+1
- check the dictionary self._freq[f+1] and create a new DLinkedList there if needed
- append the node to the DLinkedList at frequency *f+1*
- return node.val

**put(key, value)**

If key is already in cache
- do the same thing as get(key), 
- update node.val as value
Otherwise:
- if the cache is full, pop the least frequenly used element (*)
- add new node to self._node
- add new node to self._freq[1]
- reset self._minfreq to 1

(*) The least frequently used element is the tail element in the DLinkedList with frequency self._minfreq



In [ ]:
import collections

class Node:
    def __init__(self, key, val):
        self.key = key
        self.val = val
        self.freq = 1
        self.prev = self.next = None

class DLinkedList:
    """ An implementation of doubly linked list.
    
    Two APIs provided:
    
    append(node): append the node to the head of the linked list.
    pop(node=None): remove the referenced node. 
                    If None is given, remove the one from tail, which is the least recently used.
                    
    Both operation, apparently, are in O(1) complexity.
    """
    def __init__(self):
        self._sentinel = Node(None, None) # dummy node
        self._sentinel.next = self._sentinel.prev = self._sentinel
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def append(self, node):
        node.next = self._sentinel.next
        node.prev = self._sentinel
        node.next.prev = node
        self._sentinel.next = node
        self._size += 1
    
    def pop(self, node=None):
        if self._size == 0:
            return
        
        if not node:
            node = self._sentinel.prev

        node.prev.next = node.next
        node.next.prev = node.prev
        self._size -= 1
        
        return node
        
class LFUCache:
    def __init__(self, capacity):
        """
        :type capacity: int
        
        Three things to maintain:
        
        1. a dict, named as `self._node`, for the reference of all nodes given key.
           That is, O(1) time to retrieve node given a key.
           
        2. Each frequency has a doubly linked list, store in `self._freq`, where key
           is the frequency, and value is an object of `DLinkedList`
        
        3. The min frequency through all nodes. We can maintain this in O(1) time, taking
           advantage of the fact that the frequency can only increment by 1. Use the following
           two rules:
           
           Rule 1: Whenever we see the size of the DLinkedList of current min frequency is 0,
                   the min frequency must increment by 1.
           
           Rule 2: Whenever put in a new (key, value), the min frequency must 1 (the new node)
           
        """
        self._size = 0
        self._capacity = capacity
        
        self._node = dict() # key: Node
        self._freq = collections.defaultdict(DLinkedList)
        self._minfreq = 0
        
        
    def _update(self, node):
        """ 
        This is a helper function that used in the following two cases:
        
            1. when `get(key)` is called; and
            2. when `put(key, value)` is called and the key exists.
         
        The common point of these two cases is that:
        
            1. no new node comes in, and
            2. the node is visited one more times -> node.freq changed -> 
               thus the place of this node will change
        
        The logic of this function is:
        
            1. pop the node from the old DLinkedList (with freq `f`)
            2. append the node to new DLinkedList (with freq `f+1`)
            3. if old DlinkedList has size 0 and self._minfreq is `f`,
               update self._minfreq to `f+1`
        
        All of the above opeartions took O(1) time.
        """
        freq = node.freq
        
        self._freq[freq].pop(node)
        if self._minfreq == freq and not self._freq[freq]:
            self._minfreq += 1
        
        node.freq += 1
        freq = node.freq
        self._freq[freq].append(node)
    
    def get(self, key):
        """
        Through checking self._node[key], we can get the node in O(1) time.
        Just performs self._update, then we can return the value of node.
        
        :type key: int
        :rtype: int
        """
        if key not in self._node:
            return -1
        
        node = self._node[key]
        self._update(node)
        return node.val

    def put(self, key, value):
        """
        If `key` already exists in self._node, we do the same operations as `get`, except
        updating the node.val to new value.
        
        Otherwise, the following logic will be performed
        
        1. if the cache reaches its capacity, pop the least frequently used item. (*)
        2. add new node to self._node
        3. add new node to the DLinkedList with frequency 1
        4. reset self._minfreq to 1
        
        (*) How to pop the least frequently used item? Two facts:
        
        1. we maintain the self._minfreq, the minimum possible frequency in cache.
        2. All cache with the same frequency are stored as a DLinkedList, with
           recently used order (Always append at head)
          
        Consequence? ==> The tail of the DLinkedList with self._minfreq is the least
                         recently used one, pop it...
        
        :type key: int
        :type value: int
        :rtype: void
        """
        if self._capacity == 0:
            return
        
        if key in self._node:
            node = self._node[key]
            self._update(node)
            node.val = value
        else:
            if self._size == self._capacity:
                node = self._freq[self._minfreq].pop()
                del self._node[node.key]
                self._size -= 1
                
            node = Node(key, value)
            self._node[key] = node
            self._freq[1].append(node)
            self._minfreq = 1
            self._size += 1

Another Implementation, similar to the first...

In [ ]:
'''
The idea here is to maintain an OrderedDict of all keys,
where the values are: [value, frequency]

Note that reading from the LFU cache also updates the frequency.
get (key) = (3) -> 1, [1,1] --- 2, [4, 2] --- 3, [3, 2] (frequency of 3 has been updated)

I would also keep hashmap for each element -> frequency and regularly update it.

For every time I call upon a get method, I move that key to the front of the orderedList 
and update it's frequency. 
Therefore, for this updated frequency, the elements are already in order of least used.

Now if I have to append an key and the capacity is maxed out, 
then I find the minimum frequency through hashmap values. 
Find the first occurence of that frequency in ordereddict, remove that element.
'''

from collections import OrderedDict
class LFUCache:

    def __init__(self, capacity: int):
        
        # od will store the values [0] and frequencies [1] of each key
        self.od = OrderedDict()
        
        # hm will store just the frequency of each key
        self.hm = dict()

        # an integer to keep track of the max capacity of the cache
        self.maxSize = capacity
        
        

    def get(self, key: int) -> int:
        # if the key is not found in the frequency dict, then it does not exist in the cache
        if key not in self.hm:
            return -1
        
        # the first items in the od will be evicted first
        # so moving it to the end ensures that it will last longer
        self.od.move_to_end(key)
                
        self.od[key][1] += 1 #update the frequency for that key in od
        self.hm[key] += 1  #update the frequency for that key in hm
        
        # return the value from the od for that key
        return self.od[key][0]
        

    def put(self, key: int, value: int) -> None:
        
        # can't put anything in if the capacity is zero!
        if self.maxSize == 0:
            return 
        
        # if the key is already present, we need to replace an existing value
        if key in self.hm:
            
            # since we just accessed it, we move it to the end of the od so it will last longer
            self.od.move_to_end(key)
            
            self.hm[key] += 1  #update the frequency
            self.od[key][1] += 1  #update the frequency
            self.od[key][0] = value  #update the value in case value was changed
            
            # Note: all of this could be accomplished by calling:
            #self.get(key)
            
        #key is not in the dict -> two cases:
        
        # 1. we have not reached our max capacity:
        elif len(self.hm) < self.maxSize:
            
            self.od[key] = [value, 1]  #append the new key 
            self.hm[key] = 1  #append the new key 

        # 2. we have already reached our max capacity
        else:
            # time to evict something!
            
            # find the minimum frequency 
            # Note: this may be slow, because it has to scan all of the frequencies to find the minimum! O(n)
            lF = min(self.hm.values()) if len(self.hm) > 0 else None
            
            # Somehow, if there is no lowest frequency?
            if not lF:
                return 
            
            # append the new key, value (Do this after finding the minimum frequency, 
            # otherwise - it is possible your new key, value gives minimum as 1. 
            self.od[key] = [value,1]
            self.hm[key] = 1
            
            # find the first occurence of lF because for all these frequency 
            # every element is ordered in terms of least used
            for key, value in self.od.items():
                
                if value[1] == lF:
                    removeKey = key
                    break
            
            #remove that first occuerence key
            self.hm.pop(removeKey)
            del self.od[removeKey]

def see_inside(lfu):
    for key in lfu.od:
        print('key: '+str(key)+', value: '+str(lfu.od[key][0])+', frequency: '+str(lfu.od[key][1]))